## before model building

In [4]:
import pandas as pd
df = pd.read_csv("/Users/hexiaoyu/Desktop/Courses/INSY- DATA MINING/UniversalBank.csv")
df

,UserID,Personal Loan,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,1,0,25,1,49,4,1.6,1,0,1,0,0,0
1,2,0,45,19,34,3,1.5,1,0,1,0,0,0
2,3,0,39,15,11,1,1.0,1,0,0,0,0,0
3,4,0,35,9,100,1,2.7,2,0,0,0,0,0
4,5,0,35,8,45,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,0,29,3,40,1,1.9,3,0,0,0,1,0
4996,4997,0,30,4,15,4,0.4,1,85,0,0,1,0
4997,4998,0,63,39,24,2,0.3,3,0,0,0,0,0
4998,4999,0,65,40,49,3,0.5,2,0,0,0,1,0


In [5]:
df = pd.get_dummies(data = df, columns = ['Education'])
df

,UserID,Personal Loan,Age,Experience,Income,Family,CCAvg,Mortgage,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
0,1,0,25,1,49,4,1.6,0,1,0,0,0,True,False,False
1,2,0,45,19,34,3,1.5,0,1,0,0,0,True,False,False
2,3,0,39,15,11,1,1.0,0,0,0,0,0,True,False,False
3,4,0,35,9,100,1,2.7,0,0,0,0,0,False,True,False
4,5,0,35,8,45,4,1.0,0,0,0,0,1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,0,29,3,40,1,1.9,0,0,0,1,0,False,False,True
4996,4997,0,30,4,15,4,0.4,85,0,0,1,0,True,False,False
4997,4998,0,63,39,24,2,0.3,0,0,0,0,0,False,False,True
4998,4999,0,65,40,49,3,0.5,0,0,0,1,0,False,True,False


In [7]:
X = df.iloc[:,2:16]
y = df["Personal Loan"]

In [8]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5)

In [9]:
# Standardize the numerical features
from sklearn.preprocessing import MinMaxScaler
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
scaled_X_train = X_train.copy()
scaled_X_test = X_test.copy()

scaler = MinMaxScaler()
scaled_X_train[numerical_features] = scaler.fit_transform(scaled_X_train[numerical_features])
scaled_X_test[numerical_features] = scaler.transform(scaled_X_test[numerical_features])

## model building with KNN with all features

In [10]:
# Build a KNN model with k=square root (n) and evaluate
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=60)
model = knn.fit(scaled_X_train,y_train)
y_test_pred = model.predict(scaled_X_test)

from sklearn.metrics import accuracy_score
accuracy_knn = accuracy_score(y_test, y_test_pred)
accuracy_knn

0.9033333333333333

## modeling building with LASSO

In [11]:
# Run LASSO with alpha=0.01
from sklearn.linear_model import Lasso
Is = Lasso(alpha=0.01)
model = Is.fit(scaled_X_train,y_train)

In [12]:
pd.DataFrame(list(zip(scaled_X_train.columns,model.coef_)), columns = ['predictor','coefficient'])

,predictor,coefficient
0,Age,-0.000000
1,Experience,-0.000000
2,Income,0.480013
3,Family,0.013536
4,CCAvg,0.000000
5,Mortgage,0.000000
6,Securities Account,-0.000000
7,CD Account,0.149345
8,Online,-0.000000
9,CreditCard,-0.000000


In [13]:
# Fit the model on new dataset with selected features and evaluate
X_lasso_train = scaled_X_train[['Income', 'Family', 'CD Account', 'Education_1']]
X_lasso_test = scaled_X_test[['Income', 'Family', 'CD Account', 'Education_1']]
model = knn.fit(X_lasso_train,y_train)
y_test_pred = model.predict(X_lasso_test)
accuracy_knn_lasso = accuracy_score(y_test, y_test_pred)
accuracy_knn_lasso

0.9533333333333334

## model building with RF feature importance

In [14]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(random_state=0)
model = randomforest.fit(scaled_X_train,y_train)

In [15]:
feature_importance_df = pd.DataFrame(list(zip(X_train.columns,model.feature_importances_)), columns = ['predictor','feature importance'])
feature_importance_df = feature_importance_df.sort_values(by='feature importance', ascending=False)

In [16]:
feature_importance_df

,predictor,feature importance
2,Income,0.323572
4,CCAvg,0.185337
10,Education_1,0.122986
3,Family,0.110044
7,CD Account,0.049340
11,Education_2,0.043262
0,Age,0.037047
5,Mortgage,0.036724
1,Experience,0.036633
12,Education_3,0.033736


In [17]:
# Fit the model on new dataset and evaluate

X_rf_train = scaled_X_train[['Income', 'CCAvg', 'Education_1', 'Family']]
X_rf_test = scaled_X_test[['Income', 'CCAvg', 'Education_1', 'Family']]
model = knn.fit(X_rf_train,y_train)
y_test_pred = model.predict(X_rf_test)
accuracy_knn_rf = accuracy_score(y_test, y_test_pred)
accuracy_knn_rf

0.96

there is a noticeable increase of model performance in terms of the feature selection and interested in PCA's performance on this 

In [29]:
X_new=X.copy()
scaler = MinMaxScaler()
scaled_X=scaler.fit_transform(X_new)
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(scaled_X)


PCA(n_components=10)

In [30]:
pca.explained_variance_ratio_

array([0.22850422, 0.17993165, 0.1488043 , 0.13031073, 0.08835479,
       0.07806636, 0.06331542, 0.03833634, 0.02166174, 0.01485659])

In [31]:
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [32]:
model = knn.fit(X_train_pca,y_train)
y_test_pred = model.predict(X_test_pca)
accuracy_knn_pca = accuracy_score(y_test, y_test_pred)
accuracy_knn_pca

0.9033333333333333

seems like PCA's performance is not as good as the feature selection 